In [37]:
from qaqc_gis_new import QA_QC_GIS
import pandas as pd
from qaqc_gis import QA_QC_GIS as qaqcfirst

In [38]:
kern = pd.read_excel('../../Данные/qaqc/Керн1.xlsx')
las_path = '../../Данные/qaqc/ГИС/4001PL.las'
depth = kern['Глубина отбора образца после привязки к ГИС, м'].to_numpy()
poro = kern['Открытая пористость по воде, Кп, %'].to_numpy()
lithology = kern['Литологическое описание'].to_numpy()
poroeff = kern['Эффективная пористость, Кп эф, %'].to_numpy()
perm = kern['Газопроницаемость параллельно напластованию, Кпр, мД'].to_numpy()

In [41]:
las_path = '../../Данные/qaqc/ГИС/4001PL.las'
qaqcc = qaqcfirst(las_path)
qaqcc.get_list_of_tests()

FileNotFoundError: [WinError 3] Системе не удается найти указанный путь: '../Данные/qaqc/ГИС/9281PL.las'

In [30]:
list_of_tests = qaqc.get_list_of_tests()

In [31]:
qaqc.start_tests()

TypeError: start_tests() missing 1 required positional argument: 'list_of_tests'

In [7]:
list_of_tests

[]